# Analyse exploratoire des données (AED) — Pitié-Salpêtrière

Données fictives : admissions quotidiennes par service et occupation des lits.

**Objectifs** :
- Décrire les tendances d'admissions et d'occupation
- Identifier les périodes critiques et la saisonnalité
- Préparer les dataviz pour le rapport et le dashboard

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

ROOT = Path.cwd().parent if Path.cwd().name == "01-aed" else Path.cwd()
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from src.data.load import load_admissions, load_occupation

In [ ]:
admissions = load_admissions(ROOT / "data")
occupation = load_occupation(ROOT / "data")
print("Admissions:", admissions.shape)
print(admissions.head(10))
print("\nOccupation:", occupation.shape)
occupation.head()

## Statistiques descriptives

In [ ]:
adm_quotidien = admissions.groupby("date")["admissions"].sum()
print("Admissions quotidiennes (total):")
print(adm_quotidien.describe())
print("\nTaux d'occupation:")
print(occupation["taux_occupation"].describe())

## Tendances et saisonnalité

In [ ]:
occupation["mois"] = occupation["date"].dt.month
occupation["annee"] = occupation["date"].dt.year
occupation["jour_semaine"] = occupation["date"].dt.dayofweek

fig, axes = plt.subplots(2, 1, figsize=(12, 8))
occupation.plot(x="date", y="taux_occupation", ax=axes[0], legend=False)
axes[0].axhline(y=0.85, color="orange", linestyle="--", label="Seuil alerte")
axes[0].axhline(y=0.95, color="red", linestyle="--", label="Seuil critique")
axes[0].set_title("Taux d'occupation quotidien")
axes[0].legend()

adm_quotidien.plot(ax=axes[1])
axes[1].set_title("Admissions quotidiennes totales")
plt.tight_layout()
plt.show()

In [ ]:
# Répartition par service (moyenne)
by_service = admissions.groupby("service")["admissions"].sum().sort_values(ascending=False)
by_service.plot(kind="bar", figsize=(10, 4), title="Admissions totales par service")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

## Périodes critiques

Identifier les jours où le taux d'occupation dépasse les seuils (85 %, 95 %).

In [ ]:
occupation["alerte"] = "normal"
occupation.loc[occupation["taux_occupation"] >= 0.85, "alerte"] = "alerte"
occupation.loc[occupation["taux_occupation"] >= 0.95, "alerte"] = "critique"
print(occupation["alerte"].value_counts())
occupation[occupation["alerte"] != "normal"][["date", "taux_occupation", "alerte"]].head(20)

---
**Suite** : modèles statistiques dans `02-modeles-statistiques/`, modèles de prédiction dans `03-modeles-prediction/`.